In [ ]:
# en base a una hora, seleccionar los datos e ir visualizando 

In [1]:
import pandas as pd
import geopandas as gpd
import time
import datetime

In [2]:
stops = pd.read_csv('colectivos-gtfs/stops.txt').dropna(axis=1)
routes = pd.read_csv('colectivos-gtfs/routes.txt').dropna(axis=1)
trips = pd.read_csv('colectivos-gtfs/trips.txt').dropna(axis=1)
stopTimes = pd.read_csv('colectivos-gtfs/stop_times.txt').dropna(axis=1)
calendar = pd.read_csv('colectivos-gtfs/calendar.txt').dropna(axis=1)
shapes = pd.read_csv('colectivos-gtfs/shapes.txt').dropna(axis=1)
#frequencies = pd.read_csv('colectivos-gtfs/frequencies.txt').dropna(axis=1)
#https://github.com/ManushiM/CartoCamp_Workshops/blob/master/Workshop2_May2017/CartoCamp_D3.ipynb
#https://stackoverflow.com/questions/29134048/how-do-i-animate-points-down-a-path-on-a-map-in-d3-js

In [3]:
#bondi 26
linea26 = routes[routes.route_short_name==26]
linea26

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
8,60,1,26,26,Barrio Rivadavia (Capital Federal) - Retiro (C...,3
9,618,1,26,26,Barrio Rivadavia (Capital Federal) - Retiro (C...,3


In [4]:
#viajes de la linea 26
viajes26 = trips[trips.route_id==60]
viajes26.head()

,route_id,service_id,trip_id,trip_short_name,direction_id,shape_id
944,60,2,4383147,2910,0,8787
945,60,2,1765290576530,11441,0,8787
946,60,2,1765290576699,11442,0,8787
947,60,2,1765290576868,11443,0,8787
948,60,2,1765290577037,11444,0,8787


In [6]:
paradas26 = stops[stops.stop_id.isin(viaje26.stop_id)]
paradas26.to_csv('paradas26.csv')

In [8]:
#cronograma de un viaje
idViaje = viajes26.iloc[1].trip_id
viaje26 = stopTimes[stopTimes.trip_id==idViaje]
#solo arrival, departure es redundante
viaje26.arrival_time = pd.to_datetime(viaje26.arrival_time,format= '%H:%M:%S',errors='coerce',exact=True)
#traigo geodata de estaciones
viaje26 = viaje26.merge(paradas26.loc[:,['stop_id','stop_lat','stop_lon']], how = 'left')

viaje26.drop(['timepoint','trip_id','departure_time','stop_id','shape_dist_traveled','stop_sequence'],axis=1,inplace=True) 
#ordenar por tiempo y borrar repetidos
viaje26.sort_values(by=['arrival_time'], inplace=True)
duplicados = viaje26.arrival_time.duplicated(keep='first')
viaje26 = viaje26.loc[~ duplicados,:]

viaje26.head()

,arrival_time,stop_lat,stop_lon
0,1900-01-01 00:00:00,-34.587823,-58.368679
2,1900-01-01 00:02:00,-34.589636,-58.370079
3,1900-01-01 00:05:00,-34.589153,-58.372513
4,1900-01-01 00:07:00,-34.591756,-58.374527
5,1900-01-01 00:11:00,-34.596670,-58.372078


In [ ]:
viaje26['duracion'] = (viaje26.arrival_time - viaje26.arrival_time.shift(1))
viaje26['duracion'].iloc[0]=0
viaje26['duracion'] = viaje26['duracion'].map(lambda x: x.seconds)
viaje26.head()

In [ ]:
now = datetime.datetime.now()
now.minute



In [ ]:
shapeViaje26 = shapes[shapes.shape_id == viajes26.shape_id.unique()[0]]
shapeViaje26.to_csv('shapeviaje26.csv')
